# Using config files

Configuration is a huge problem for machine learning code, because you may want to expose almost any detail of any function as a hyperparameter. The setting you want to expose might be arbitrarily 
far down in your call stack. Default values also become hard to change without breaking backwards compatibility.

To solve this problem, Thinc provides a config system that lets you easily describe arbitrary trees of objects. The objects can be created via function calls you register using a simple decorator syntax. The config can include values like hyperparameters or training settings (whatever you need), or references to functions and the values of their arguments.

Thinc will parse the configuration file as a dict and fill in the references to values defined in other sections.
For example, `${hyper_params:learn_rate}` is substituted with `0.001` in the next cell.

Keys starting with `@` are references to **registered functions**.



In [1]:
from thinc.api import Config, registry

EXAMPLE_CONFIG1 = """
[hyper_params]
learn_rate = 0.001

[optimizer]
@optimizers = "Adam.v1"
learn_rate = ${hyper_params:learn_rate}
"""

config1 = Config().from_str(EXAMPLE_CONFIG1)
config1

{'hyper_params': {'learn_rate': 0.001},
 'optimizer': {'@optimizers': 'Adam.v1', 'learn_rate': 0.001}}

Configs can also define **nested blocks** using the `.` notation. In this example, `optimizer.learn_rate` defines the `learn_rate` argument of the `optimizer` block. Instead of a float, the learning rate can also be a generator – for instance, a linear warm-up rate:

In [2]:
EXAMPLE_CONFIG2 = """
[optimizer]
@optimizers = "Adam.v1"

[optimizer.learn_rate]
@schedules = "warmup_linear.v1"
initial_rate = 2e-5
warmup_steps = 1000
total_steps = 10000
"""

config2 = Config().from_str(EXAMPLE_CONFIG2)
config2

{'optimizer': {'@optimizers': 'Adam.v1',
  'learn_rate': {'@schedules': 'warmup_linear.v1',
   'initial_rate': 2e-05,
   'warmup_steps': 1000,
   'total_steps': 10000}}}

Thinc will create the optimizer and pass in the schedule as the `learn_rate` argument.
Each element of the `optimizer` object is accessible via `get()` method or using `[]` notation. For example:


In [21]:
print(f"initial learn rate: {config2['optimizer'].get('learn_rate').get('initial_rate')}")
print(f"initial learn rate: {config2['optimizer']['learn_rate'].get('initial_rate')}")
print(f"initial learn rate: {config2['optimizer']['learn_rate']['initial_rate']}")

initial learn rate: 2e-05
initial learn rate: 2e-05
initial learn rate: 2e-05
